In [8]:
#import relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pyarrow as pa
import seaborn as sns
from datetime import timedelta, datetime

dir = Path(r"S:\Fackler_OSS_364376\data\IRB-364376-v1-230215")

In [9]:
fp = dir.joinpath('EHR', 'd_flo_measures.csv.gz')
dict = pd.read_csv(fp, compression="gzip")

names = ["State Behavioral Scale",
"-3 Unresponsive", 
"-2 Responsive to noxious stimuli", 
"-1 Responsive to gentle touch or voice",
"0 Awake and Able to calm",
"+1 Restless and difficult to calm",
"+2 Agitated",
"State Behavioral Scale (SBS)"]

# note: flowsheet record flow_meas_id as meas_id
# note: SBS score values are only stored in these fields

fmid = [304080016, 304080017, 304080018, 304080019, 304080020, 304080021]

In [11]:
# connect with feather file
fp = dir.joinpath('EHR', 'ptsd_record.csv.gz')

ptsd_record = pd.read_csv(fp, compression="gzip")

# load flow table of all patient EHR records
fp = dir.joinpath('EHR', 'flowsheet.csv.gz')
data = pd.read_csv(fp, compression="gzip")
data = data.drop(columns = ['meas_comment', 'meas_template_id'])
# Note: pandas took 50 seconds to load the table. Consider porting to PySpark RDD

sbs = data[data['meas_id'].isin(fmid)]
# print(sbs.shape)
# 25878 entries

# calculate sbs score from offset
sbs['SBS'] = sbs['meas_id'] - 304080019
sbs = sbs.drop(columns=['meas_value', 'meas_id'])
sbs['recorded_time'] = pd.to_datetime(sbs['recorded_time'], format='%Y-%m-%d %H:%M:%S')
sbs_indiv = sbs.groupby('pat_enc_csn_sid')

# load pre-selected patients from patient_inclexcl.ipynb
patients = np.load('./DONOTPUSH/patients_wodrugs.npy', allow_pickle=True)

c:\Program Files\miniconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Segment pstd and SBS data for ML training

Select Relevant vitals.
For now:
- drop data from Medibus (ventilator system)
- Keep dts, HR, SPO2, RR, BP

In [12]:
fp_hl7m = dir.joinpath('ptsd-phi', 'vitals-hl7m', "003", '1000000003-2016-07-07-0.0166667-1-HL7M.feather')
fp_tsdb = dir.joinpath('ptsd-phi', 'vitals-tsdb', "106", '1000002106-2019-01-22-1-TSDB.feather')
fp_gevital = dir.joinpath('ptsd-phi', 'vitals-sb', "672", '1000002672-2020-10-01-1-GEVITAL.feather')
fp_medibus = dir.joinpath('ptsd-phi', 'vitals-sb', "672", '1000002672-2020-10-01-1-MEDIBUSVITAL.feather')

df = pd.read_feather(fp_hl7m, columns=None, use_threads=True, storage_options=None)
names_hl7m = df.columns.tolist()
print("hl7m", names_hl7m)
df = pd.read_feather(fp_tsdb, columns=None, use_threads=True, storage_options=None)
names_tsdb = df.columns.tolist()
print("tsdb", names_tsdb)
df = pd.read_feather(fp_gevital, columns=None, use_threads=True, storage_options=None)
names_gevitals = df.columns.tolist()
print("gevitals", names_gevitals)
# df = pd.read_feather(fp_medibus, columns=None, use_threads=True, storage_options=None)
# names_medibus = df.columns.tolist()
# print("medibus", names_medibus)

hl7m ['dts', 'AR1-D', 'AR1-M', 'AR1-R', 'AR1-S', 'CVP2', 'HR', 'NBP-D', 'NBP-M', 'NBP-R', 'NBP-S', 'PVC', 'RR', 'SPO2-%', 'SPO2-R', 'ST-AVF', 'ST-AVL', 'ST-AVR', 'ST-I', 'ST-II', 'ST-III', 'ST-V1', 'TP1-1', 'TP1-2']
tsdb ['dts', 'HR', 'HR_Art_2930', 'HR_SpO2_7876', 'Temp', 'awRR_3495', 'N2O_Insp_2885', 'N2O_Exp_2886', 'O2_Insp_2902', 'O2_Exp', 'Des_Insp', 'Des_Exp_594', 'Sevo_Insp', 'Sevo_Exp_596', 'SpO2_7874', 'ECG_I_ST', 'ECG_II_ST', 'ECG_III_ST', 'FIO2_2343', 'PIP_1227', 'PEEP_1418', 'NIBP_S', 'ABP_S_2318', 'ABP_D_2319', 'ABP_M_2320', 'CO2_Insp_perc', 'EtCO2_2905', 'NIBP_D', 'NIBP_M']
gevitals ['dts', 'PARM_HR', 'PARM_PVC', 'PARM_ST_SEG1', 'PARM_ST_SEG2', 'PARM_ST_SEG3', 'PARM_ST_V1', 'PARM_ST_V2', 'PARM_ST_V3', 'PARM_ST_V4', 'PARM_ST_V5', 'PARM_ST_V6', 'PARM_ST_AVF', 'PARM_ST_AVL', 'PARM_ST_AVR', 'PARM_SPO2_1', 'PARM_SPO2_HR1', 'PARM_NBP_CUFF', 'PARM_NBP_SYS', 'PARM_NBP_MEAN', 'PARM_NBP_DIA', 'GE_HSDI_SIGNAL_ID_NON_INVASIVE_PRESS_pulseRate_NUM', 'PARM_RESP_RATE', 'GE_HSDI_SIGNAL_ID

In [13]:
hl7m_order = ['dts', 'HR', 'ST-I', 'ST-II', 'ST-III', 'NBP-S', 'NBP-D', 'RR', 'SPO2-%']
tsdb_order = ['dts', 'HR', 'ECG_I_ST', 'ECG_II_ST', 'ECG_III_ST', 'NIBP_S','NIBP_D','awRR_3495', 'SpO2_7874']
ge_order = ['dts','PARM_HR', 'PARM_ST_SEG1', 'PARM_ST_SEG2', 'PARM_ST_SEG3', 'PARM_NBP_SYS','PARM_NBP_DIA','PARM_RESP_RATE', 'PARM_SPO2_1']
metrics = set(hl7m_order + tsdb_order + ge_order)

assert(len(hl7m_order) == len(tsdb_order))
assert(len(ge_order) == len(tsdb_order))
dims = len(hl7m_order)

In [68]:
# turn off error message on .loc
pd.options.mode.chained_assignment = None  # default='warn'

# vitals data
X = []
# sbs scores
y = []
# record patient id for stratification
ids = []

for p in patients[0:50]:
    print(p)

    files = ptsd_record[ptsd_record['pat_enc_csn_sid'] == p]
    files['start_time'] = pd.to_datetime(files['start_time'], format='%Y-%m-%d %H:%M:%S')
    files['end_time'] = pd.to_datetime(files['end_time'], format='%Y-%m-%d %H:%M:%S')
    files.sort_values('start_time')

    devices = files['device']
    filename = files['filename'] + '.feather'
    startime = files['start_time']

    dfs = []
    size = 0

    for (d, fn, t0) in zip(devices, filename, startime): 
        # drop 'MEDIBUSVITAL' since it is a ventilator (we dont want ventilated patients)
        if fn.endswith('MEDIBUSVITAL.feather'):
            continue
        # HL7M', 'TSDB', 'GEVITAL'
        if (d.endswith('HL7M') or d.endswith('TSDB')):
            fp_device = 'vitals-' + d.lower()
        else:
            fp_device = 'vitals-sb'
        fp_p = str(p)[-3:] # last 3 digit of pat_enc_csn_sid is the subfolder
        fp = dir.joinpath('ptsd-phi', fp_device, fp_p, fn)
        if (fp == None):
            print(fp, "does not exist")
            continue

        df = pd.read_feather(fp, columns=None, use_threads=True, storage_options=None)
        df = df.filter(metrics)
        # rearrange columns according to device
        if d.endswith('HL7M'):
            df = df.reindex(columns=hl7m_order)
        elif d.endswith('TSDB'):
            df = df.reindex(columns=tsdb_order)
        elif d.endswith('GEVITAL'):
            df = df.reindex(columns=ge_order)

        df.loc[:,'dts'] = pd.to_timedelta(df.loc[:,'dts'], unit='s')
        df.loc[:,'dts'] = df.loc[:,'dts'] + t0

        # standardize column names
        # time, heart rate, ST segments, non-invasive blood pressure, RR, SPO2
        df.columns = ['dts', 'HR', 'ST-I', 'ST-II', 'ST-III', 'NBP-S', 'NBP-D', 'RR', 'SPO2-%']
        dfs.append(df)

    # resample to 60s intervals using median value
    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.resample.html
    vitals = (pd.concat(dfs, axis=0)
              .set_index('dts')
              .sort_values(by=['dts'])
              .resample('60S').median()
    )
    # add SBS
    sbs_p = (sbs_indiv.get_group(p)
            .sort_values('recorded_time')
            .drop(columns=['osler_sid', 'pat_enc_csn_sid'])
            .set_index('recorded_time')
            .resample('60S').median()
    )
    # preserve vitals timestamps while adding SBS where available
    patient_multi = pd.merge(left=vitals, right=sbs_p, left_index=True, right_index=True, how='left')

    for i, row in patient_multi.iterrows():
        # where SBS score is found, extract the previous 30 minutes
        if pd.notna(row['SBS']):
            start = i-timedelta(seconds=30*60)
            if start in patient_multi.index:
                dat = patient_multi.loc[start:i, :]
                X.append(dat.drop(columns=['SBS']).to_numpy())
                y.append(row['SBS'])
                ids.append(p)

X = np.array(X)
y = np.array(y)
ids = np.array(ids)

1000001694
(31, 9)
(31, 9)
(31, 9)
1000001708
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
1000001711
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
1000001719
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
1000001728
(31, 9)
1000001756
(31, 9)
(31, 9)
(31, 9)
(31, 9)
1000001806
(31, 9)
(31, 9)
(31, 9)
(31, 9)
1000001857
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
1000001860
(31, 9)
(31, 9)
1000001986
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
1000001988
(31, 9)
1000001995
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
(31, 9)
1000

In [ ]:
# reshape
X = np.transpose(X, (0, 2, 1))

print(X.shape)
print(y.shape)
print(ids.shape)

(197, 8, 31)
(197,)
(197,)


In [ ]:
np.save('./DONOTPUSH/waveforms_60min.npy',X)
np.save('./DONOTPUSH/sbs_60min.npy',y)
np.save('./DONOTPUSH/ids_60min.npy',ids)